In [2]:
import torch
import torchsummary
import netron

from models.prediciton.CVCNet.models.mmoe import MMoE, ML_MMoE
from models.prediciton.CVCNet.models.cvcnet import CVCNet, ExtractionNet
from models.prediciton.CVCNet.models.submodules import DNN, MLPBlock
from models.prediciton.CVCNet.models.base_mtl import BaseMTL
from models.prediciton.CVCNet.models.split import SplitMTL

In [2]:
!pwd

/workspace/src/RDCVC


In [4]:
def count_parameters(model, only_trainable=False):
    """Count the number of parameters in a model.

    Args:
        model (torch.nn.Module): The model to count the parameters of.
        only_trainable (bool, optional): Whether to count only trainable parameters.
            Defaults to False.

    Returns:
        int: The number of parameters in the model.
    """
    if only_trainable:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())

## SplitMTL

In [10]:
split = SplitMTL(
    inputs_dim=18,
    target_dict={"AIRFLOW": 4, "PRESENCE": 6},
    bottom_units=[4096, 4096, 4096],
    activation="leakyrelu:0.1",
)
print(count_parameters(split))
# print(basemtl)
input_dummy = torch.randn(1, 18)
model_traced = torch.jit.trace(split, input_dummy)
model_traced.save("split.pt")
netron.start("split.pt")

167931914


## BaseMTL

In [4]:
basemtl = BaseMTL(
    inputs_dim=18,
    target_dict={"AIRFLOW": 4, "PRESENCE": 6},
    bottom_units=[10, 15, 20],
    tower_units=[88],
    activation="leakyrelu:0.1",
)
print(count_parameters(basemtl))
# print(basemtl)
input_dummy = torch.randn(1, 18)
model_traced = torch.jit.trace(basemtl, input_dummy)
model_traced.save("basemtl.pt")
netron.start("basemtl.pt")

5261
Serving 'basemtl.pt' at http://localhost:8080


/usr/local/lib/python3.10/site-packages/torch/jit/_trace.py:1056: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(


('localhost', 8080)

## CVCNet

In [14]:
cvcNet = CVCNet(
    inputs_dim=18,
    target_dict={"AIRFLOW": 4, "PRES": 6},
    num_layers=2,
    num_tasks_experts=1,
    num_shared_experts=2,
    expert_units=[10, 15, 20],
    expert_activation="leakyrelu:0.1",
    expert_dropout_rate=None,
    expert_use_bn=False,
    tower_units=[11, 16],
    tower_activation="leakyrelu:0.1",
    tower_dropout_rate=None,
    tower_use_bn=None,
)
print(count_parameters(cvcNet))
# input_dummy = torch.randn(1, 18)
# model_traced = torch.jit.trace(cvcNet, input_dummy)
# model_traced.save("cvcNet.pt")
# netron.start("cvcNet.pt")

9746


## MMoE

In [7]:
mmoe = MMoE(
    inputs_dim=18,
    target_dict={"PRESENCE": 6, "AIRFLOW": 4},
    num_experts=3,
    expert_units=[10,15,20],
    expert_activation="leakyrelu:0.1",
    expert_dropout_rate=None,
    expert_use_bn=False,
    tower_units=[11, 16, 21],
    tower_activation="leakyrelu:0.1",
    tower_dropout_rate=None,
    tower_use_bn=None,
    is_last_layer=True,
)

torchsummary.summary(mmoe, (18,),device="cpu")

x = torch.randn(1, 18)
model_path = "./demo.pth"  # 定义模型数据保存的路径
torch.onnx.export(mmoe, x, model_path)  # 将 pytorch 模型以 onnx 格式导出并保存
# print(mmoe)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             190
         LeakyReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 15]             165
         LeakyReLU-4                   [-1, 15]               0
            Linear-5                   [-1, 20]             320
         LeakyReLU-6                   [-1, 20]               0
               DNN-7                   [-1, 20]               0
            Expert-8                   [-1, 20]               0
            Linear-9                   [-1, 10]             190
        LeakyReLU-10                   [-1, 10]               0
           Linear-11                   [-1, 15]             165
        LeakyReLU-12                   [-1, 15]               0
           Linear-13                   [-1, 20]             320
        LeakyReLU-14                   

In [20]:
eNet = ExtractionNet(
    inputs_dim=18,
    target_dict={"PRESENCE": 6, "AIRFLOW": 4},
    num_tasks_experts=1,
    num_shared_experts=2,
    expert_units=[10,15,20],
    expert_activation="leakyrelu:0.1",
    expert_dropout_rate=None,
    expert_use_bn=False,
    tower_units=[11, 16, 21],
    tower_activation="leakyrelu:0.1",
    tower_dropout_rate=None,
    tower_use_bn=None,
    is_last_layer=True,
)

torchsummary.summary(eNet, (18,),device="cpu")

x = torch.randn(1, 18)
model_path = "../../eNet_demo.pth"  # 定义模型数据保存的路径
torch.onnx.export(eNet, x, model_path)  # 将 pytorch 模型以 onnx 格式导出并保存
# print(mmoe)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             190
         LeakyReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 15]             165
         LeakyReLU-4                   [-1, 15]               0
            Linear-5                   [-1, 20]             320
         LeakyReLU-6                   [-1, 20]               0
               DNN-7                   [-1, 20]               0
            Expert-8                   [-1, 20]               0
            Linear-9                   [-1, 10]             190
        LeakyReLU-10                   [-1, 10]               0
           Linear-11                   [-1, 15]             165
        LeakyReLU-12                   [-1, 15]               0
           Linear-13                   [-1, 20]             320
        LeakyReLU-14                   

/usr/local/lib/python3.10/site-packages/torch/jit/_trace.py:1056: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(


def forward(self,
    inputs: Tensor) -> List[Tensor]:
  extractionNet_layers = self.extractionNet_layers
  _1 = getattr(extractionNet_layers, "1")
  extractionNet_layers0 = self.extractionNet_layers
  _0 = getattr(extractionNet_layers0, "0")
  _2, _3, _4, _5, = (_0).forward(inputs, )
  _6, _7, = (_1).forward(_2, _3, _4, _5, )
  return [_6, _7]

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [10]:
import os
os.chdir("/workspace/src/")
!pwd

/workspace/src


In [14]:
model = torch.load("/workspace/checkpoints/NN/cvcnet-mtl-mlp_18_3_16_32_512:512:512_256:256:256_BS128_LR0.0002_EP1000_2023-12-04T02-40-49/final_model.pth")

In [28]:
import numpy as np
# 定义总参数量、可训练参数量及非可训练参数量变量
Total_params = 0
Trainable_params = 0
NonTrainable_params = 0

# 遍历model.parameters()返回的全局参数列表
for param in model.parameters():
    mulValue = np.prod(param.size())  # 使用numpy prod接口计算参数数组所有元素之积
    Total_params += mulValue  # 总参数量
    if param.requires_grad:
        Trainable_params += mulValue  # 可训练参数量
    else:
        NonTrainable_params += mulValue  # 非可训练参数量

print(f'Total params: {Total_params}')
print(f'Trainable params: {Trainable_params}')
print(f'Non-trainable params: {NonTrainable_params}')

Total params: 170426986
Trainable params: 170426986
Non-trainable params: 0


In [34]:
count_parameters(model)

170426986

# Generate data based on MAE and RMSE

In [20]:
a=1
b=5
c=7
e=2
l = [a,b,c,e]
s=sum(l)
l=[i/s for i in l]
print(l,sum(l))

[0.06666666666666667, 0.3333333333333333, 0.4666666666666667, 0.13333333333333333] 1.0


In [25]:
A=2958271
b=2958212
torch.nn.Softmax(dim=0)(torch.tensor([A,b],dtype=torch.float32))

tensor([1.0000e+00, 2.3803e-26])